In [1]:
# import physics_environments
import numpy as np
from physics_environments.envs.rl_pendulum.cart_pendulum_math.robotic_eq_solutions import robotic_equation_n1
from physics_environments import CartPendulumPhysicsConstants, CartPendulumPhysicsEnvironmentParams, CartPendulumPhysicsEnvironment, \
                                 RLCartPendulumEnvironment, RLCartPendulumEnvironmentParams

from jumanji import register, registered_environments
from jumanji.wrappers import VmapWrapper, AutoResetWrapper, VmapAutoResetWrapper
import jumanji

import jax
import jax.numpy as jnp
import time

In [2]:
n = 1
constants = CartPendulumPhysicsConstants(n    = n,         
                                         g    = 9.81,        
                                         l    = n*[0.300],
                                         r    = n*[0.200],
                                         m    = n*[0.800],
                                         I    = n*[0.011],
                                         mu   = n*[0.015])

env_type             = 'compound pendulum'
cart_pendulum_params = CartPendulumPhysicsEnvironmentParams(constants       = constants,         
                                                            env_type        = env_type,      
                                                            save_after_init = False, 
                                                            save_filename   = f"env_{n}n_{env_type}_obj.pkl") 

rl_env_params = RLCartPendulumEnvironmentParams(physics_env_params          = cart_pendulum_params,
                                                training_generator_type     = "DefaultGenerator",
                                                dt                          = 0.010,
                                                t_episode                   = 10.00,
                                                max_cart_acceleration       = 25.00,
                                                cart_width                  = 0.250,
                                                track_width                 = 0.800
                                                )

rl_env = RLCartPendulumEnvironment(params = rl_env_params)

In [3]:
rl_env.action_spec

BoundedArray(shape=(1,), dtype=dtype('float32'), name='cart acceleration', minimum=Array(-25., dtype=float32), maximum=Array(25., dtype=float32))

In [5]:
key = jax.random.key(seed=42)
state, timestep           = rl_env.reset(key)
next_state, next_timestep = rl_env.step(state = state, action=jnp.array([3]))

In [5]:
assert ('cart_n_pendulum-v1' in registered_environments())

"""
Register the environment in the outermost __init__.py with:

register(id="cart_n_pendulum-v1", entry_point="physics_environments.envs.rl_pendulum:RLCartPendulumEnvironment")
""";

In [6]:
env = jumanji.make('cart_n_pendulum-v1')
key = jax.random.PRNGKey(0)
state, timestep = env.reset(key)
state, timestep = env.step(state=state, action=jnp.array([0]))

In [7]:
def rollout(batch_size=2, rollout_length=3):
    # Note: setting device=jax.devices('cpu') as default argument actually hurts performance slightly, having device without defualt argument helps to fix this.

    def step_func(state, key):
        action = jax.random.randint(key=key, minval=0, maxval=num_actions, shape=(1))
        new_state, timestep = env.step(state, action)
        return new_state, timestep

    def rollout_func(state, key, n):
        random_keys = jax.random.split(key, n)
        state, rollout = jax.lax.scan(step_func, state, random_keys)
        return rollout

    # Constants
    num_actions = env.action_spec.shape[0]

    # Create random Keys
    master_key = jax.random.PRNGKey(0)
    key1, key2 = jax.random.split(master_key) # create a separate random key for step() and reset()

    # Instantiate a batch of environment states (via vmap reset)
    keys = jax.random.split(key1, batch_size)
    state, timestep = jax.vmap(env.reset)(keys)

    # Collect a batch of rollouts (via vmap rollout_func)
    keys = jax.random.split(key2, batch_size)
    rollout = jax.vmap(rollout_func, in_axes=(0, 0, None))(state, keys, rollout_length)
    return jnp.array(rollout.observation)
    
def robotic_equation_n1_numpy(t, y, args):
    theta_1, dtheta_1, ddx_c  = y
    g, r_1, m_1, mu_1, I_1    = args
    return np.array([dtheta_1,
                      (-dtheta_1*mu_1 + ddx_c*m_1*r_1*np.cos(theta_1) + g*m_1*r_1*np.sin(theta_1))/(I_1 + m_1*r_1**2),
                      ddx_c
                    ])

In [8]:
# rollout(batch_size=2, rollout_length=3)

Array([[[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
           2.7500001e-01,  3.1415927e+00, -3.1855762e-08,
          -3.1855761e-06, -8.7422777e-08,  3.1855762e-08,
           3.1855761e-06, -1.0000000e+00, -2.7849192e-15,
          -2.7747712e-13,  2.6226834e-08, -9.5567287e-09,
          -9.5567293e-07, -3.0000001e-01, -8.3547577e-16,
          -8.3243145e-14,  3.0000001e-01, -9.5567293e-17,
          -9.5567305e-15],
         [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
           2.7500001e-01,  3.1415927e+00, -6.3600588e-08,
          -3.1744826e-06, -8.7422777e-08,  6.3600588e-08,
           3.1744826e-06, -1.0000000e+00, -5.5601398e-15,
          -2.7347707e-13,  2.6226834e-08, -1.9080177e-08,
          -9.5234481e-07, -3.0000001e-01, -1.6680421e-15,
          -8.2043117e-14,  3.0000001e-01, -1.9080179e-16,
          -9.5234489e-15],
         [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
           2.7500001e-01,  3.1415927e+00, -9.5234860e-08,
          -3.16342

In [8]:
gpu = jax.devices("gpu")[0]  # First GPU
cpu = jax.devices("cpu")[0]  # First CPU
t=0
y=np.array([1,2,3])
args=np.array([1,2,3,4,5])

t_1024=np.array([[0]]*1024).T
y_1024=np.array([[[1,2,3]]*1024]).T
args_1024=np.array([[[1,2,3,4,5]]*1024]).T
rollout_jax_jit = jax.jit(rollout, static_argnames=['batch_size', 'rollout_length'])

### Tests for n=1

#### Single Environment, 10000 Rollouts

---

In [17]:
t = time.time()
with jax.default_device(cpu): 
    rollout(1,10000).block_until_ready()
time.time()-t

0.33071374893188477

In [18]:
t = time.time()
with jax.default_device(gpu):
    rollout(1,10000).block_until_ready()
time.time()-t

21.558382987976074

In [14]:
t = time.time()
for i in range(10000):
    robotic_equation_n1_numpy(t=t, y=y, args=args)
time.time()-t

0.058182477951049805

---

#### 1024 Environments, 10000 Rollouts

---

In [20]:
t = time.time()
with jax.default_device(cpu):
    rollout(1024,10000).block_until_ready()
time.time()-t

4.405309677124023

In [21]:
t = time.time()
with jax.default_device(gpu):
    rollout(1024,10000).block_until_ready()
time.time()-t

21.132590293884277

In [15]:
t = time.time()
for i in range(10000):
    robotic_equation_n1_numpy(t=t_1024, y=y_1024, args=args_1024)
time.time()-t

0.59600830078125

- GPUs times are not affected by bigger batch-sizes as GPUs always run fully parallel

---

Let's just jit the jax loops:

In [37]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1024,10001).block_until_ready()
time.time()-t

3.6228606700897217

In [38]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1024,10001).block_until_ready()
time.time()-t

3.2823872566223145

In [26]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1024,10001).block_until_ready()
time.time()-t

21.488381385803223

In [27]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1024,10001).block_until_ready()
time.time()-t

20.391295671463013

- Good speed-up for the CPU, and slight speedup for the GPU for a lot of iterations.
- Note that the compilation time on the first call was lower than not compiling at all

---

#### 10240 Environments, 10000 Rollouts

---

In [49]:
t = time.time()
with jax.default_device(cpu):
    rollout(10240,1000).block_until_ready()
time.time()-t

4.091904163360596

In [50]:
t = time.time()
with jax.default_device(gpu):
    rollout(10240,1000).block_until_ready()
time.time()-t

3.379101037979126

---

Let's just jit the jax loops:

In [51]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(10240,1000).block_until_ready()
time.time()-t

3.251147747039795

In [52]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(10240,1000).block_until_ready()
time.time()-t

3.0582923889160156

In [53]:
t = time.time() 
with jax.default_device(gpu):
    rollout_jax_jit(10240,1000).block_until_ready()
time.time()-t

1.9264230728149414

In [54]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(10240,1000).block_until_ready()
time.time()-t

1.6756553649902344

- The CPU might be the fastest for simple simulations with n=1 for batch-sizes of 1024
- However large GPUs as the Nvidia A100 80 GB will give extraordinary speeds, compared to the available RTX 2080 Ti 16 GB.
- But the GPU will surpass the CPU in speed if the vectorization further increases, i.e. batch-sizes of 10240 or internal matrix calculations.
    - The pendulum is mostly a single CPU-heavy function and solving steps without much internal parallelization

### Tests for n=4

In [9]:
n = 4
constants = CartPendulumPhysicsConstants(n    = n,         
                                         g    = 9.81,        
                                         l    = n*[0.300],
                                         r    = n*[0.200],
                                         m    = n*[0.800],
                                         I    = n*[0.011],
                                         mu   = n*[0.015])

env_type             = 'compound pendulum'
cart_pendulum_params = CartPendulumPhysicsEnvironmentParams(constants       = constants,         
                                                            env_type        = env_type,      
                                                            save_after_init = False, 
                                                            save_filename   = f"env_{n}n_{env_type}_obj.pkl") 

rl_env_params = RLCartPendulumEnvironmentParams(physics_env_params          = cart_pendulum_params,
                                                training_generator_type     = "DefaultGenerator",
                                                dt                          = 0.010,
                                                t_episode                   = 10.00,
                                                max_cart_acceleration       = 25.00,
                                                cart_width                  = 0.250,
                                                track_width                 = 0.800
                                                )

rl_env = RLCartPendulumEnvironment(params = rl_env_params)

<class 'physics_environments.envs.rl_pendulum.cart_pendulum_math.types.CartPendulumPhysicsConstants'> {'n': 4, 'g': 9.81, 'l': [0.3, 0.3, 0.3, 0.3], 'r': [0.2, 0.2, 0.2, 0.2], 'm': [0.8, 0.8, 0.8, 0.8], 'I': [0.011, 0.011, 0.011, 0.011], 'mu': [0.015, 0.015, 0.015, 0.015]}


In [10]:
def robotic_equation_n4_numpy(t, y, args):
    theta_1, theta_2, theta_3, theta_4, dtheta_1, dtheta_2, dtheta_3, dtheta_4, ddx_c                    = y
    g, l_1, l_2, l_3, r_1, r_2, r_3, r_4, m_1, m_2, m_3, m_4, mu_1, mu_2, mu_3, mu_4, I_1, I_2, I_3, I_4 = args

    return np.array([dtheta_1,
                      dtheta_2,
                      dtheta_3,
                      dtheta_4,

                      (dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1) - l_1*m_4*r_4*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))*np.cos(theta_4 - theta_1)/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))))*np.cos(theta_3 - theta_1)/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))))*np.cos(theta_2 - theta_1)/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2),

                      (dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2),

                      (dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)),

                      (-dtheta_4*mu_4 - dtheta_3**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_3*mu_4 - dtheta_2**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) - dtheta_1**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + ddx_c*m_4*r_4*np.cos(theta_4) + g*m_4*r_4*np.sin(theta_4) - l_1*m_4*r_4*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))*(dtheta_4**2*l_3*m_4*r_4*np.sin(theta_4 - theta_3) + dtheta_4*mu_4 - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + m_3*r_3)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_4**2*l_2*m_4*r_4*np.sin(theta_4 - theta_2) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + m_2*r_2)*(dtheta_4**2*l_1*m_4*r_4*np.sin(theta_4 - theta_1) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2)))/(I_4 - l_1**2*m_4**2*r_4**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + m_4*r_4**2 - (-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2) - (-l_1**2*m_4*r_4*(l_3*m_4 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3*m_4*r_4*np.cos(theta_4 - theta_3) - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_4*r_4*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*m_4*r_4*np.cos(theta_4 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_3**2*m_4 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2*(l_3*m_4 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + m_2*r_2**2))),
                      ddx_c
                    ])

In [11]:
env = jumanji.make('cart_n_pendulum-v1', params=rl_env_params)
key = jax.random.PRNGKey(0)
state, timestep = env.reset(key) # takes long for n=5
#state, timestep = env.step(state=state, action=jnp.array([0])) # takes long for n=5

<class 'physics_environments.envs.rl_pendulum.cart_pendulum_math.types.CartPendulumPhysicsConstants'> {'n': 4, 'g': 9.81, 'l': [0.3, 0.3, 0.3, 0.3], 'r': [0.2, 0.2, 0.2, 0.2], 'm': [0.8, 0.8, 0.8, 0.8], 'I': [0.011, 0.011, 0.011, 0.011], 'mu': [0.015, 0.015, 0.015, 0.015]}


In [12]:
t    = np.array([0])
y    = np.array([1, 2, 3, 4, 11, 22, 33, 44, 1])
args = np.array([9.81  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  ,
       0.2  , 0.8  , 0.8  , 0.8  , 0.8, 0.015, 0.015, 0.015,
       0.015, 0.011, 0.011, 0.011, 0.011])

t_1024    = np.array([[0]]*1024).T
y_1024    = np.array([[[1, 2, 3, 4, 11, 22, 33, 44, 1]]*1024]).T
args_1024 = np.array([[[9.81  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  ,
       0.2  , 0.8  , 0.8  , 0.8  , 0.8, 0.015, 0.015, 0.015,
       0.015, 0.011, 0.011, 0.011, 0.011]]*1024]).T

#### Single Environment, 1 Rollout

---

In [30]:
t = time.time()
with jax.default_device(cpu): 
    rollout(1,1).block_until_ready()
time.time()-t

20.5720157623291

In [31]:
t = time.time()
with jax.default_device(gpu):
    rollout(1,1).block_until_ready()
time.time()-t

12.168337106704712

In [29]:
t = time.time()
for i in range(1):
    robotic_equation_n4_numpy(t=t, y=y, args=args)
time.time()-t

0.0022623538970947266

- Numpy with a proper solver takes around 15-25 ms from prior testing. This numpy runs just with the robotic equation, and not the full step function.

---

Let's just jit the function:

In [51]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1,1).block_until_ready()
time.time()-t

15.980771541595459

In [54]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1,1).block_until_ready()
time.time()-t

0.0007648468017578125

In [52]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1,1).block_until_ready()
time.time()-t

10.691083669662476

In [53]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1,1).block_until_ready()
time.time()-t

0.0031943321228027344

- After compiling, jax actually seems to be faster than numpy on simple inference tasks

#### 1024 Environments, 1 Rollout

---

In [20]:
t = time.time()
with jax.default_device(cpu):
    rollout(1024,1).block_until_ready()
time.time()-t

4.405309677124023

In [21]:
t = time.time()
with jax.default_device(gpu):
    rollout(1024,1).block_until_ready()
time.time()-t

21.132590293884277

In [ ]:
t = time.time()
for i in range(1):
    robotic_equation_n4_numpy(t=t_1024, y=y_1024, args=args_1024)
time.time()-t

0.042040109634399414

- Numpy scales better as the batch size increases, but struggles with slow python loops
- GPUs times are not affected by bigger batch-sizes as GPUs always run fully parallel

---

Let's just jit the jax loops:

In [36]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1024,1).block_until_ready()
time.time()-t

17.48338270187378

In [37]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1024,1).block_until_ready()
time.time()-t

0.0036160945892333984

In [38]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1024,1).block_until_ready()
time.time()-t

10.661686897277832

In [39]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1024,1).block_until_ready()
time.time()-t

0.0042459964752197266

In [49]:
t = time.time()
for i in range(1000):
    robotic_equation_n4_numpy(t=t, y=y, args=args)
time.time()-t

2.066396951675415

- Much better: compiling is a have to with long functions. For many iterations, this will be a speed up

---

#### 10240 Environments, 10000 Rollouts

In [46]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1024,10000).block_until_ready()
time.time()-t

38.82838273048401

In [47]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1024,10000).block_until_ready()
time.time()-t

24.740278720855713

In [44]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1024,10000).block_until_ready()
time.time()-t

22.510165214538574

In [45]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1024,10000).block_until_ready()
time.time()-t

26.011308908462524

In [41]:
t = time.time()
for i in range(10000):
    robotic_equation_n4_numpy(t=t_1024, y=y_1024, args=args_1024)
time.time()-t

341.30730080604553

- There is a good speed-up with JAX, while it even has to solve the robotic equation.
- Jax was able to simulate 1.024.000 pendulums steps (equal to ~1024 episodes) in 24 s on a RTX 2080 Ti (12 GB), or an AMD Ryzen 5 3600x CPU (16 GB RAM).
    - Numpy took way longer for the simple function, while JAX would also enable a more seamless end-to-end GPU training.
    - More GPU memory will help for faster training.

---

#### 1 Environment, 10000 Rollouts

In [19]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1,10000).block_until_ready()
time.time()-t

13.441340923309326

In [49]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_jit(1,10000).block_until_ready()
time.time()-t

0.04753279685974121

In [10]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1,10000).block_until_ready()
time.time()-t

35.87016463279724

In [11]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_jit(1,10000).block_until_ready()
time.time()-t

21.76958656311035

In [43]:
t = time.time()
for i in range(10000):
    robotic_equation_n4_numpy(t=t, y=y, args=args)
time.time()-t

19.53590154647827

- Also the loop step time is a lot faster than numpy, which is important for evaluations.
- The CPU performs very well on unbatched inputs.
- However, the GPU speed wont change upon changing the batch-size.
    - --> a CPU might be faster for simple benchmarks than a GPU
    - --> however one can perform multiple similar benchmarks concurrently to save time and to get more accurate benchmark statistics.

---

### Is pmap as fast as jit?

Since pmap automatically jits its functions, it would be interesting to know if it is also as fast as jit.

In [13]:
def rollout_pmappable(non_static_input, batch_size=2, rollout_length=3): # pmap needs at least one non_static argument to function
    # Note: setting device=jax.devices('cpu') as default argument actually hurts performance slightly, having device without defualt argument helps to fix this.

    def step_func(state, key):
        action = jax.random.randint(key=key, minval=0, maxval=num_actions, shape=(1,))
        new_state, timestep = env.step(state, action)
        return new_state, timestep

    def rollout_func(state, key, n):
        random_keys = jax.random.split(key, n)
        state, rollout = jax.lax.scan(step_func, state, random_keys)
        return rollout

    # Constants
    num_actions = env.action_spec.shape[0]

    # Create random Keys
    master_key = jax.random.PRNGKey(0)
    key1, key2 = jax.random.split(master_key) # create a separate random key for step() and reset()

    # Instantiate a batch of environment states (via vmap reset)
    keys = jax.random.split(key1, batch_size)
    state, timestep = jax.vmap(env.reset)(keys)

    # Collect a batch of rollouts (via vmap rollout_func)
    keys = jax.random.split(key2, batch_size)
    rollout = jax.vmap(rollout_func, in_axes=(0, 0, None))(state, keys, rollout_length)
    return jnp.array(rollout.observation)+non_static_input

In [14]:
non_static_input_cpu = jnp.arange(jax.device_count(), device=cpu) + 0.05
non_static_input_gpu = jnp.arange(jax.device_count(), device=gpu) + 0.05

rollout_jax_pmap_jit_cpu = jax.pmap(rollout_pmappable, static_broadcasted_argnums=[1,2], backend='cpu') #, axis_name="device"
rollout_jax_pmap_jit_gpu = jax.pmap(rollout_pmappable, static_broadcasted_argnums=[1,2], backend='gpu') #, axis_name="device"
rollout_jax_pmap_jit_gpu

In [15]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_pmap_jit_cpu(non_static_input_cpu, 1, 10000).block_until_ready()
time.time()-t

20.58776068687439

In [16]:
t = time.time()
with jax.default_device(cpu):
    rollout_jax_pmap_jit_cpu(non_static_input_cpu, 1,10000).block_until_ready()
time.time()-t

0.06576824188232422

In [17]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_pmap_jit_gpu(non_static_input_gpu, 1,10000).block_until_ready()
time.time()-t

35.839993953704834

In [18]:
t = time.time()
with jax.default_device(gpu):
    rollout_jax_pmap_jit_gpu(non_static_input_gpu, 1,10000).block_until_ready()
time.time()-t

24.119675397872925

- All compilation times of pmap appear to be slightly higher aswell as the run times.
- But it is still acceptable.
    - --> One can convert jax.pmap on multi-device code to jax.jit for single-device code to get some small speed-up